# Prompt Engineering and Chat Completion
This is a simple tutorial that teaches prompt engineering basics and analyzes Twitter data using OpenAI large language models (LLMs).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it securely. This tutorial uses [AWS Secrets Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLMs predict the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates the quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [ ]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- openai: call the OpenAI APIs.

In [ ]:
pip install openai -q

Load the OpenAI API key and define a `openai_help` function.

In [ ]:
from openai import OpenAI
from IPython.display import display, Image, Markdown
openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [ ]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

Add a system message asking LLM to act as a high school teacher with different temperatures.

In [ ]:
messages = [
    {"role": "system", "content": "use tone as a middle school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

Add assistant messages to teach LLM what `##` is.

In [ ]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [ ]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

## Analyze Twitter data

In [ ]:
import json

with open('tweet_text.json', 'r', encoding='utf-8') as f:
    tweet_data = json.load(f)

In [ ]:
print("Number of tweets:", len(tweet_data))
print("Sample tweet:", tweet_data[0])

### Summarization 
- Analyze tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [ ]:
messages = [
    {"role": "system", "content": f"""provide a summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

In [ ]:
messages = [
    {"role": "system", "content": f"""provide a summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

In [ ]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people learn AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [ ]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [ ]:
for tweet in tweet_data:
    # print(tweet['tweet']['text'])
    flag_help(tweet['tweet']['text'])

### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [ ]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

In [ ]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract the mentioned people's or organization's names
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [ ]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned people or organizations;
                                        step 4 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]
    print(openai_help(messages))

In [ ]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

### Expanding with multiple prompts 
- Identify topics
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [ ]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative, or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned people or organizations;
                                        step 4 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


In [ ]:
print(analysis_result)

In [ ]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis result delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets mentioned by people or organizations;
                                        step 2 {delimiter} identify the common sentiments and emotions for each mentioned person or organization;
                                        step 3 {delimiter} Organize the result in a json document with keys <people or organization name> 
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

## Create a chatbot

In [ ]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chatbot that answers user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people or organization name in the {delimiter}{analysis_summary}{delimiter}, report the corresponding sentiment and emotion
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    display(Markdown(f"Chatbot: {reply}"))

## 🧠 Try It Yourself: Prompt Engineering Practice
👉 Be sure to **run your own cells** and show the outputs before uploading the notebook to GitHub.

1. **Summarize your tweets differently**  
   Change your system message to summarize tweets from a new perspective —  
   for example:  
   *“Summarize what people say about AI in education.”*  
   or  
   *“Summarize how people feel about the election.”*

2. **Translate tweets**  
   Write a prompt that translates 10 tweets into another language,  
   such as Chinese, Spanish, or French.


3. **Simplify or rewrite tweets**  
   Ask the model to rewrite 10 tweets in a more polite, formal, or funny tone.

4. **Find keywords**  
   Design a prompt that extracts the top five most common keywords  
   from your tweet text and returns them as a list.

## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
